In [2]:
%defaultDatasource jdbc:h2:file:./database/ILOSTATdata.db

# Destruir tabelas

In [ ]:

DROP TABLE IF EXISTS POPULATION_PROP;
DROP TABLE IF EXISTS POPULATION_TOTAL;
DROP TABLE IF EXISTS UNEMPLOYMENT;
DROP TABLE IF EXISTS POVERTY;
DROP TABLE IF EXISTS POPULATION;

# Carregar dados brutos

In [27]:
DROP TABLE IF EXISTS UNEMPLOYMENT;
DROP TABLE IF EXISTS POVERTY;
DROP TABLE IF EXISTS POPULATION;

CREATE TABLE POPULATION(
    LOCAL VARCHAR(50),
    IDADE_CODE VARCHAR(50),
    ANO INTEGER,
    VALOR DOUBLE,
    PRIMARY KEY(LOCAL,IDADE_CODE,ANO)
) AS SELECT 
    ref_area,
    classif1,
    time,
    obs_value
FROM CSVREAD('./datasets/ILOSTAT_CHRIS/population.csv');


CREATE TABLE POVERTY(
    LOCAL VARCHAR(50),
    IDADE_CODE VARCHAR(50),
    ANO INTEGER,
    VALOR DOUBLE,
    PRIMARY KEY(LOCAL,IDADE_CODE,ANO)
) AS SELECT 
    ref_area,
    classif1,
    time,
    obs_value
FROM CSVREAD('./datasets/ILOSTAT_CHRIS/poverty.csv');



CREATE TABLE UNEMPLOYMENT(
    LOCAL VARCHAR(50),
    IDADE_CODE VARCHAR(50),
    ANO INTEGER,
    VALOR DOUBLE,
    PRIMARY KEY(LOCAL,IDADE_CODE,ANO)
) AS SELECT 
    ref_area,
    classif1,
    time,
    obs_value
FROM CSVREAD('./datasets/ILOSTAT_CHRIS/unemployment.csv');


In [28]:
SELECT * FROM POPULATION;
SELECT * FROM POVERTY;
SELECT * FROM UNEMPLOYMENT;

### Calcular população relativa ao total

In [48]:
DROP TABLE IF EXISTS POPULATION_PROP;
DROP TABLE IF EXISTS POPULATION_TOTAL;

CREATE VIEW POPULATION_TOTAL AS
SELECT LOCAL, ANO, VALOR
    FROM POPULATION
    WHERE IDADE_CODE='AGE_AGGREGATE_TOTAL';

CREATE VIEW POPULATION_PROP AS
SELECT P1.LOCAL, P1.ANO, P1.IDADE_CODE, P1.VALOR, (P1.VALOR / P2.VALOR) AS PROPORCAO
    FROM POPULATION P1, POPULATION_TOTAL P2
    WHERE P1.LOCAL = P2.LOCAL AND P1.ANO = P2.ANO;

In [49]:
SELECT * FROM POPULATION_PROP;

### Processar indicadores

In [65]:
DROP TABLE IF EXISTS ILOSTAT;

CREATE VIEW ILOSTAT AS
SELECT PV.LOCAL, PV.ANO, PV.VALOR AS POBREZA_15_MAIS, UN.VALOR AS DESEMPREGO_15_64
    FROM
        POVERTY PV, UNEMPLOYMENT UN
    WHERE PV.LOCAL = UN.LOCAL AND PV.ANO = UN.ANO AND PV.IDADE_CODE = 'AGE_YTHADULT_YGE15' AND UN.IDADE_CODE = 'AGE_YTHADULT_Y15-64'
    ORDER BY ANO, LOCAL;
    
SELECT * FROM ILOSTAT